In [48]:
#import libraries
import os
import pandas as pd

In [49]:
#Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
#Look through all folders in the "Project" folder to access dataset
folder_path = '/content/drive/MyDrive/Project'

data = []

for root, dirs, files in os.walk(folder_path):
    for filename in files:
        if filename.endswith('.txt'):
            file_path = os.path.join(root, filename)
            date = os.path.splitext(filename)[0]  # Extract date from filename
            try:
                df = pd.read_csv(file_path, delimiter='\s+', skiprows=[0, 2], encoding='latin1')
                df = df.iloc[:, :16]
                df.columns = ['Time', 'Wind Dir', 'Wind Spd', 'Hum In', 'Humidity', 'Temp In', 'Temp', 'Raw Barom', 'Temp Ch1', 'Hum Ch1', 'Temp Ch2', 'Hum Ch2', 'UV', 'Solar', 'Dew Point', 'Rain Rate']
                df.insert(0, 'Date', pd.to_datetime(date, format='%y%m%d').strftime('%d-%m-%y'))

                data.append(df)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

Error processing file /content/drive/MyDrive/Project/Dataset_3/120825.txt: No columns to parse from file
Error processing file /content/drive/MyDrive/Project/Dataset_3/121220.txt: Error tokenizing data. C error: Expected 30 fields in line 689, saw 31



In [54]:
# Concatenate all data into a single DataFrame
data = pd.concat(data, ignore_index=True)

# Convert 'date' and 'Time' columns to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%y')
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time
data.sort_values(by=['Date', 'Time'], inplace=True)
data.set_index('Date', inplace=True)

print(data.head())

                Time  Wind Dir  Wind Spd  Hum In  Humidity  Temp In  Temp  \
Date                                                                        
2011-08-05  12:51:00     220.0       2.0    37.0      66.0     27.2  21.3   
2011-08-05  12:52:00     170.0       5.0    36.0      65.0     27.2  21.3   
2011-08-05  12:53:00     171.0       6.0    36.0      64.0     27.3  21.4   
2011-08-05  12:54:00     186.0       8.0    36.0      65.0     27.3  21.4   
2011-08-05  12:55:00     204.0       6.0    36.0      66.0     27.3  21.4   

            Raw Barom  Temp Ch1  Hum Ch1  Temp Ch2  Hum Ch2   UV  Solar  \
Date                                                                      
2011-08-05     1006.0       0.0      0.0       0.0      0.0  6.0  749.0   
2011-08-05     1006.0       0.0      0.0       0.0      0.0  6.1  788.0   
2011-08-05     1006.0       0.0      0.0       0.0      0.0  6.3  810.0   
2011-08-05     1006.0       0.0      0.0       0.0      0.0  6.3  721.0   
2011-08-05

In [52]:
pd.set_option('display.max_info_columns', 100)
pd.set_option('display.max_info_rows', 10000000)
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6513084 entries, 2011-08-05 to 2024-05-21
Data columns (total 16 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Time       6513084 non-null  object 
 1   Wind Dir   6513084 non-null  float64
 2   Wind Spd   6513084 non-null  float64
 3   Hum In     6513084 non-null  float64
 4   Humidity   6513084 non-null  float64
 5   Temp In    6513084 non-null  float64
 6   Temp       6513084 non-null  float64
 7   Raw Barom  6513084 non-null  float64
 8   Temp Ch1   6513084 non-null  float64
 9   Hum Ch1    6513084 non-null  float64
 10  Temp Ch2   6513084 non-null  float64
 11  Hum Ch2    6513084 non-null  float64
 12  UV         6513084 non-null  float64
 13  Solar      5434359 non-null  float64
 14  Dew Point  714608 non-null   float64
 15  Rain Rate  714608 non-null   float64
dtypes: float64(15), object(1)
memory usage: 844.7+ MB
